### Different Scenarios

Run imports:

In [2]:
from roads import load_road

import visualizer
from visualizer import plot_with_bounds
%matplotlib notebook
%matplotlib inline


road = load_road('./data/straight.pkl')
print(road.get_road_position(7.7142857142857135, 115.57142857142857))
#
# N = 100
# plot_with_bounds('n', [
#     [
#        road.n_min(i/N * road.length),
#        road.n_max(i/N * road.length)
#     ] for i in range(N)
# ],
#                  x_values=[i/N * road.length for i in range(N)],
#                  no_bounds=True
#                  )
#
# min(road.n_max(x, road_segment_idx=1) for x in np.linspace(0, 1, 1000))
# for i in range(10):
#     print(road.get_tangent_angle_at(i * road.length/10))
    # print(road.get_global_position(i * road.length/10, 0))

(0.0, -1.0)


#### Benchmarks

In [ ]:
import benchmark
from path_planner import Objectives
%matplotlib notebook
%matplotlib inline
# TODO: solver currently adds v >= 3 as a constraint on the states
cfg = benchmark.config
benchmarks = benchmark.run(
    benchmark.BenchmarkConfiguration(
        start_velocities=[
            5,
            10,
            20,
        ],
        start_offset=cfg.LateralOffset.Mid,
        velocity_range=(0.7, 1),
        roads=[
            cfg.Road.Straight,
            cfg.Road.Left_Turn,
            cfg.Road.Lane_Change,
            cfg.Road.Slalom,
            cfg.Road.ElchTest_One,
            cfg.Road.Feasible_Curve,
            cfg.Road.Infeasible_Curve,
            cfg.Road.Random,
        ],
        time_horizon=cfg.TimeHoriozon.Medium,
        time_discretization=cfg.TimeDiscretization.PlateauLinear(1/50, 1/60, 5),
        models=[
            # (cfg.Model.PointMassModel, cfg.SolverType.Convex),
            (cfg.Model.BicycleModel, cfg.SolverType.Convex),
        ],
        objectives=[
            # Objectives.maximize_distance,
            # Objectives.minimize_offset_from_reference_path,
            Objectives.minimize_control_derivatives,
            Objectives.minimize_control_derivatives_offset_maximize_distance
        ],
        replanning_steps=5
    ),
)



for benchmark in benchmarks:
    benchmark.save_animation()
    benchmark.save_stats()
    benchmark.save_to_csv()
    # -----------------------
    # display(benchmark.get_animation())
    # benchmark.car.predictive_model.plot_additional_information([
    #     state for state, _ in benchmark.car.predictive_model_states
    # ], [
    #     control for control, _ in benchmark.car.predictive_model_controls
    # ])
    # benchmark.plot_car_states()
    # benchmark.plot_predictive_car_states()
    # benchmark.plot_controls()
    # benchmark.plot_predictive_controls()
    # benchmark.car.plot_metrics()

# rsync -avh ~/bachelor-thesis/code/benchmark-results/* andreas@wg-server.net:~/files



Progress: [█-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------] 0.69%  of the Road Complete, current state: Global Position x: 8.95, Global Postion y: 116.57, Steering Angle: -0.00, Velocity: 4.90, Orientation: -0.00, Yaw Rate: -0.00, Slip Angle: -0.00, planned next 83.33ms in 8.07ms          

#### Small-Angle-Approximation

In [13]:
import numpy as np
from visualizer import plot_with_bounds
from math import radians

from self_driving_cars import DynamicSingleTrackModel
from models import PointMassModel, BicycleModel, AbstractVehicleModel
from roads import load_road
from path_planner import NonConvexPathPlanner
%matplotlib notebook
%matplotlib inline


road = load_road('./data/left_turn.pkl')
dt = 1/100

kst = BicycleModel(
    road=road,
    v_range=(0, 10),
    acc_range=(-6, 3),
    steering_angle_range=(radians(-30), radians(30)),
    steering_velocity_range=(-2, 2),
    l_wb=0.883+1.508,
)

point_mass = PointMassModel(
    road=road,
    v_x_range=(0, 10),
    v_y_range=(-4, 4),
    acc_x_range=(-4, 4),
    acc_y_range=(-4, 4),
    yaw_rate_range=(-4, 4),
    yaw_acc_range=(-4, 4),
    a_max=8,
)

velocity = 2
x, y = road.get_global_position(0, 0)
# 0: Global Position x,
# 1: Global Postion y,
# 2: Steering Angle,
# 3: Velocity,
# 4: Orientation,
# 5: Yaw Rate,
# 6: Slip Angle
car_initial_state = np.array([
        x, y, 0, velocity, road.get_tangent_angle_at(0), 0, 0
    ])

car = DynamicSingleTrackModel(
    predictive_model=point_mass,  # won't be used
    initial_state=car_initial_state,
    planner= NonConvexPathPlanner(point_mass, dt, 1, lambda:_),# won't be used
    velocity_range=(0, 10),
    acceleration_range=(-4, 4),
    steering_range=(radians(-30), radians(30)),
    steering_velocity_range=(-2, 2),
    road=road,
)

point_mass_initial_state = point_mass.get_state_vec_from_dsm(car_initial_state)

kst_cur = kst.get_state_vec_from_dsm(car_initial_state)
pm_cur = point_mass_initial_state
car_cur = car_initial_state

y_values_list = [
        [
            point_mass.convert_vec_to_state(
                (pm_cur, pm_cur := point_mass.forward_euler_step(
                    last_pm := pm_cur,
                    np.array([0, 0]),
                    dt)[0])[0]
            ).get_position_orientation()[-1],
            (car_cur, car_cur := car._update(
                car_cur,
                point_mass.get_dsm_control_from_vec(
                    np.array([0, 0]),
                    last_pm,
                    None,
                    dt,
                    None,
                    AbstractVehicleModel.CarState(car_cur[2], car_cur[4]),
                ),
                dt
                )
            )[0][4]
        ] for _ in range(1000)
    ]
# print(y_values_list)
plot_with_bounds(
    y_values_list=y_values_list,
    y_labels=[
        'Point mass orientation',
        'Car Orientation'
    ],
    y_label="",
    no_bounds=True,
    store_as_pgf=True,
    pgf_name='lag_orientation.pgf',
)


#### Visualize McCormick

In [4]:
import visualizer
x_bounds=(-2, 3)
y_bounds=(-10, 10)

visualizer.visualize_mccormick(x_bounds, y_bounds)
visualizer.visualize_mccormick_2d_interactive(x_bounds, y_bounds)

2025-02-19 13:45:42.014 Python[72369:12893793] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-19 13:45:42.014 Python[72369:12893793] +[IMKInputSession subclass]: chose IMKInputSession_Modern


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='y_fixed', max=10.0, min=-10…